In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('slow',20),('fast',5),('sig',3)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Setting up the minimal requirements for Bollinger Bands indicator
        # Or one can use the inbuilt BollingerBands indicatore provided by BT
        self.simpma=bt.indicators.SimpleMovingAverage(self.data,period=self.p.slow)
        self.std=bt.indicators.StdDev(self.simpma,period=12,safepow=True)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        print()

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            if  self.simpma < self.data.close: 
                # Using the very basic SMA < market price to enter the trade

                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if (self.data.low > self.simpma+2*self.std or self.data.low < self.simpma-2*self.std):
                                # Using upper Bollinger Band to fix a suitable exit position
                                # and lower band for stoploss
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()                
                


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    
    datapath = ('TSLA.csv')

    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        dtformat=('%Y-%m-%d'),
        tmformat=('%H.%M.%S'),
       
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00
2019-10-28, BUY CREATE, 65.54
2019-10-29, BUY EXECUTED, Price: 64.00, Cost: 6399.80, Comm 0.00
2019-10-29, SELL CREATE, 63.24
2019-10-30, SELL EXECUTED, Price: 62.60, Cost: 6399.80, Comm 0.00
2019-10-30, OPERATION PROFIT, GROSS -139.80, NET -139.80

2019-10-30, BUY CREATE, 63.00
2019-10-31, BUY EXECUTED, Price: 62.62, Cost: 6262.00, Comm 0.00
2019-10-31, SELL CREATE, 62.98
2019-11-01, SELL EXECUTED, Price: 63.26, Cost: 6262.00, Comm 0.00
2019-11-01, OPERATION PROFIT, GROSS 64.40, NET 64.40

2019-11-01, BUY CREATE, 62.66
2019-11-04, BUY EXECUTED, Price: 62.96, Cost: 6296.00, Comm 0.00
2019-11-04, SELL CREATE, 63.49
2019-11-05, SELL EXECUTED, Price: 63.92, Cost: 6296.00, Comm 0.00
2019-11-05, OPERATION PROFIT, GROSS 96.40, NET 96.40

2019-11-05, BUY CREATE, 63.44
2019-11-06, BUY EXECUTED, Price: 63.60, Cost: 6360.00, Comm 0.00
2019-11-06, SELL CREATE, 65.32
2019-11-07, SELL EXECUTED, Price: 65.83, Cost: 6360.00, Comm 0.00
2019-11-07, OPERATION PROFIT, 

2020-06-16, SELL CREATE, 196.43
2020-06-17, SELL EXECUTED, Price: 197.54, Cost: 18355.80, Comm 0.00
2020-06-17, OPERATION PROFIT, GROSS 1398.40, NET 1398.40

2020-06-17, BUY CREATE, 198.36
2020-06-18, BUY EXECUTED, Price: 200.60, Cost: 20060.00, Comm 0.00
2020-06-18, SELL CREATE, 200.79
2020-06-19, SELL EXECUTED, Price: 202.56, Cost: 20060.00, Comm 0.00
2020-06-19, OPERATION PROFIT, GROSS 195.60, NET 195.60

2020-06-19, BUY CREATE, 200.18
2020-06-22, BUY EXECUTED, Price: 199.99, Cost: 19999.00, Comm 0.00
2020-06-22, SELL CREATE, 198.86
2020-06-23, SELL EXECUTED, Price: 199.78, Cost: 19999.00, Comm 0.00
2020-06-23, OPERATION PROFIT, GROSS -21.40, NET -21.40

2020-06-23, BUY CREATE, 200.36
2020-06-24, BUY EXECUTED, Price: 198.82, Cost: 19882.20, Comm 0.00
2020-07-01, SELL CREATE, 223.93
2020-07-02, SELL EXECUTED, Price: 244.30, Cost: 19882.20, Comm 0.00
2020-07-02, OPERATION PROFIT, GROSS 4547.40, NET 4547.40

2020-07-02, BUY CREATE, 241.73
2020-07-06, BUY EXECUTED, Price: 255.34, Cost: 